### Problem statment

Let's suppose, that there are a grup of objects - hunters. The goal of each hunter is to be inside of circle with given radius of group center and aloso do not touch other hunters (retain your own comfort zone). 
Hunter problem is that he knows restricted amount of information:

1. Dirrection of closest hunter velocity $v_{c}$: $\frac{v_{c}}{||v_{c}||}$
2. Does hunter distance to closest hunter reduced: $\{0,1\}$
3. Does closest hunter break into comfort zone with specified radius: $\{0,1\}$
4. Dirrection of group velocity $v_{g}$: $\frac{v_{g}}{||v_{g}||}$
5. Does hunter distance to group center reduced: $\{0,1\}$
6. Is hunter inside the circle of group center: $\{0,1\}$
7. Dirrection of hunter velocity $v_{h}$: $\frac{v_{h}}{||v_{h}||}$
8. Hunter force: $f_{h}$

In descrete time case hunter will have the following information:

1. Dirrection of closest hunter velocity: $\frac{x_{h_{c}}^0 - x_{h_{c}}^{-1}}{||x_{h_{c}}^0 - x_{h_{c}}^{-1}||}$

    $ c = arg \min\limits_{k\ne i} \left|\left|x_{h_i}^{-1}-x_{h_k}^{-1}\right|\right|$

2. Does hunter distance to closest neighbour hunter reduced: $h\left(d_i^{0}- d_i^{1}\right)$

    $ d_i^{-1} = \min\limits_{k\ne i} \left|\left|x_{h_i}^{-1}-x_{h_k}^{-1}\right|\right|$

    $ d_i^{0} = \min\limits_{k\ne i} \left|\left|x_{h_i}^{0}-x_{h_k}^{0}\right|\right|$

3. Does closest hunter break into comfort zone with radius $r_{cz}$: $h \left(r_{cz} -\left|\left|x_{h}^0-x_{c}^0\right|\right| \right)$

4. Dirrection of group velocity: $\frac{x_{g}^0 - x_{g}^{-1}}{||x_{g}^0 - x_{g}^{-1}||}$

    $x_g^{-1} = \overline{x_h}^{-1}$

    $x_g^{0} = \overline{x_h}^{0}$

5. Does hunter distance to group center reduced: $h\left(\left|\left|x_{h}^{-1}-x_{g}^{-1}\right|\right|-\left|\left|x_{h}^0-x_{g}^0\right|\right|\right)$

6. Is hunter inside  $r_{gc}$ - radius the circle  of the group center: $h \left(r_{gc} -\left|\left|x_{h}^0-x_{g}^0\right|\right| \right)$

7. Dirrection of hunter velocity: $\frac{x_{h_i}^0 - x_{h_i}^{-1}}{||x_{h_i}^0 - x_{h_i}^{-1}||}$

8. Hunter force: $f_{h_i}^{-1}$

### Environment

#### Hunter policy

Since hunter may moove only with speed constrained by some constant, we need to
constrain action space. Let's define max speed as $v_{max}$.
We may define our distribution via some function $G(x)$ from simple normal distribution $\mathcal{N}(x|\mu,diag(e^{2 s}))$.
Let's define function $g(x)$ so, that it was constrained in circle with radius $v_{max}$:
$$
G(x) = v_{max}\frac{x}{||x||_2}e^{-||x||_2^2} 
$$

$$\nabla_x\left[\textbf{f}(x)g(x)\right] = \frac{d\textbf{f}(x)}{dx}g(x)+\frac{dg(x)}{dx} \textbf{f}(x)^T $$

$$ \nabla_x G(x) =v_{max} \left( \frac{e^{-||x||_2^2}}{||x||_2} I + \nabla_x\left[\frac{e^{-||x||_2^2}}{||x||_2}\right]x^T\right) $$

$$ \nabla_x\left[\frac{e^{-||x||_2^2}}{||x||_2}\right] = \nabla_{||x||_2}\left[\frac{e^{-||x||_2^2}}{||x||_2}\right]\nabla_x||x||_2$$

$$\nabla_x||x||_2 = \frac{x}{||x||_2}$$
$$  \nabla_{||x||_2}\left[\frac{e^{-||x||_2^2}}{||x||_2}\right] = -\left[\frac{1}{||x||_2^2}+2\right]e^{-||x||_2^2}$$

$$ \nabla_x G(x) =v_{max}\frac{e^{-||x||_2^2}}{||x||_2} \left(I -\left[\frac{1}{||x||_2^2}+2\right] x x^T\right)  $$

$$\mathcal{J}_{G(x)}=-\left(v_{max} \frac{e^{-||x||_2^2}}{||x||_2}\right)^n \left(1-\left[\frac{1}{||x||_2^2}+2\right]||x||_2^2 \right) = 2\left(v_{max} \frac{e^{-||x||_2^2}}{||x||_2}\right)^n ||x||_2^2 $$

for 2 dimentions:
$$\mathcal{J}_{G(x)}=2 v_{max}^2 e^{-2||x||_2^2}$$

$$\ln(\mathcal{J}_{G(x)}^{-1})=2\left(||x||_2^2-\ln v_{max}\right)-\ln 2$$

So, PDF of distribution rv G(x) may be estimated as following:
$$\mathcal{N}(x|\mu,diag(e^{2s}))J_{G(x)}^{-1}$$

#### Reward assignmend

Reword is equal to:

    if current hunter inside group center circle:
        gc_reward = 2

    else, if distance to group center reduced:
        gc_reward = 1
   
    else:
        gc_reward = 0
        
    if  closest hunter break into comfort zone:
        cz_penalty = 1
    else:
        cz_penalty = 0
    
    reward = gc_reward - cz_penalty

In [110]:
import numpy as np


import numpy as np
policy = HunterPolicy(env,max_action=3)
env = GroupedClusteringHunterEnvironment(num_hunters=10)
env.state
policy(env.state)
action,_ = policy.sample_action()
env.step(action)
env.reset()

In [111]:
import torch
from policy import HunterPolicy, VictimPolicy, Baseline
# from environment import GroupedClusteringHunterEnvironment

from train import train, visualize_group
%matplotlib inline

env = GroupedClusteringHunterEnvironment(num_hunters=100)

baseline = Baseline(env)
policy = HunterPolicy(env,max_action=3)

In [112]:
%matplotlib inline


train(policy,env,100000, learning_rate=0.0001, baseline=baseline,gamma = 0.99)

ValueError: only one element tensors can be converted to Python scalars

In [3]:
policy = torch.load('grouped_force_policy_max_force_3_num_hunters_10.p')

In [5]:
torch.save(policy,'new_force_policy_ellipse_max_action_3.p')
policy = torch.load('new_force_policy_ellipse_max_action_3.p')

### Test Visualization

In [3]:
from grouped_clusteting_visualization import model_hunter_learning
%matplotlib notebook

             

model_hunter_learning('Test. max action = 3',policy,env)

ModuleNotFoundError: No module named 'grouped_clusteting_visualization'